# Building our First RAG bot - Skill: talk to Search Engine

We have now all the building blocks to build our first Bot that "talks with my data". These blocks are:

1) A well indexed hybrid (text and vector) engine with my data in chunks -> Azure AI Search
2) A good LLM python framework to build LLM Apps -> LangChain
3) Quality OpenAI GPT models that understand language and follow instructions -> GPT3.5 and GPT4
4) A persisten memory database -> CosmosDB

We are missing just one thing: **Agents**.

In this Notebook we introduce the concept of Agents and we use it to build or first RAG bot.

In [2]:
import random
import asyncio
from typing import Dict, List
from concurrent.futures import ThreadPoolExecutor
from typing import Optional, Type

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import ConfigurableField, ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

#custom libraries that we will use later in the app
from common.utils import  GetDocSearchResults_Tool
from common.prompts import AGENT_DOCSEARCH_PROMPT

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

from dotenv import load_dotenv
load_dotenv("credentials.env")


True

In [3]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

## Introducing: Agents

The implementation of Agents is inspired by two papers: the [MRKL Systems](https://arxiv.org/abs/2205.00445) paper (pronounced ‘miracle’ 😉) and the [ReAct](https://arxiv.org/abs/2210.03629) paper.

Agents are a way to leverage the ability of LLMs to understand and act on prompts. In essence, an Agent is an LLM that has been given a very clever initial prompt. The prompt tells the LLM to break down the process of answering a complex query into a sequence of steps that are resolved one at a time.

Agents become really cool when we combine them with ‘experts’, introduced in the MRKL paper. Simple example: an Agent might not have the inherent capability to reliably perform mathematical calculations by itself. However, we can introduce an expert - in this case a calculator, an expert at mathematical calculations. Now, when we need to perform a calculation, the Agent can call in the expert rather than trying to predict the result itself. This is actually the concept behind [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins).

In our case, in order to solve the problem "How do I build a smart bot that talks to my data", we need this REACT/MRKL approach, in which we need to instruct the LLM that it needs to use 'experts/tools' in order to read/load/understand/interact with a any particular source of data.

Let's create then an Agent that interact with the user and uses a Tool to get the information from the Search engine.

#### 1. We start first defining the Tool/Expert

Tools are functions that an agent can invoke. If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objectives you give it. If you don't describe the tools well, the agent won't know how to use them properly.

In [4]:
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
index3_name = "cogsrch-index-books"
indexes = [index1_name, index2_name, index3_name]

We have to convert the Retreiver object into a Tool object ("the expert"). Check out the Tool `GetDocSearchResults_Tool` in `utils.py`

Declare the tools the agent will use

In [5]:
tools = [GetDocSearchResults_Tool(indexes=indexes, k=5, reranker_th=1, sas_token=os.environ['BLOB_SAS_TOKEN'])]

#### 3. Define the LLM to use

In [6]:
COMPLETION_TOKENS = 1500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True)

#### 4. Bind tools to the LLM

Newer OpenAI models (1106 and newer) have been fine-tuned to detect when one or more function(s) should be called and respond with the inputs that should be passed to the function(s). In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call these functions. The goal of the OpenAI tools APIs is to more reliably return valid and useful function calls than what can be done using a generic text completion or chat API.

OpenAI termed the capability to invoke a single function as **functions**, and the capability to invoke one or more functions as [**tools**](https://platform.openai.com/docs/guides/function-calling).

> OpenAI API has deprecated functions in favor of tools. The difference between the two is that the tools API allows the model to request that multiple functions be invoked at once, which can reduce response times in some architectures. It’s recommended to use the tools agent for OpenAI models.

Having an LLM call multiple tools at the same time can greatly speed up agents whether there are tasks that are assisted by doing so. Thankfully, OpenAI models versions 1106 and newer support parallel function calling, which we will need to make sure our smart bot is performant.

##### **From now on and for the rest of the notebooks, we are going to use OpenAI tools API tool call our experts/tools**

To pass in our tools to the agent, we just need to format them to the [OpenAI tool format](https://platform.openai.com/docs/api-reference/chat/create) and pass them to our model. (By bind-ing the functions, we’re making sure that they’re passed in each time the model is invoked.)

In [7]:
llm_with_tools = llm.bind_tools(tools)

# Let's also add the option to configure in real time the model we want

llm_with_tools = llm_with_tools.configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt35",
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True),
)

#### 2. Define the System Prompt

Because OpenAI Function Calling is finetuned for tool usage, we hardly need any instructions on how to reason, or how to output format. We will just have two input variables: `question` and `agent_scratchpad`. `question` should be a string containing the user objective. `agent_scratchpad` should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.

Get the prompt to use `AGENT_DOCSEARCH_PROMPT` - you can modify this in `prompts.py`! Check it out!
It looks like this:

```python
AGENT_DOCSEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", CUSTOM_CHATBOT_PREFIX  + DOCSEARCH_PROMPT_TEXT),
        MessagesPlaceholder(variable_name='history', optional=True),
        ("human", "{question}"),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)
```

In [8]:
prompt = AGENT_DOCSEARCH_PROMPT

#### 5. Create the agent

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [9]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "question": lambda x: x["question"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

Or , which is equivalent, LangChain has a class that does exactly the cell code above: `create_openai_tools_agent`

```python
agent = create_openai_tools_agent(llm, tools, prompt)
```

Create an agent executor by passing in the agent and tools

In [10]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

Give it memory - since AgentExecutor is also a Runnable class, we do the same with did on Notebook 5

In [11]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

Because cosmosDB needs two fields (an id and a partition), and RunnableWithMessageHistory takes by default only one identifier for memory (session_id), we need to use `history_factory_config` parameter and define the multiple keys for the memory class

In [12]:
userid_spec = ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        )
session_id = ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        )

In [13]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[userid_spec,session_id]
)

In [14]:
# configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
config

{'configurable': {'session_id': 'session214', 'user_id': 'user893'}}

#### 6.Run the Agent!

In [15]:
%%time
agent_with_chat_history.invoke({"question": "Hi, I'm Pablo Marin. What's yours"}, config=config)

CPU times: user 196 ms, sys: 11.4 ms, total: 208 ms
Wall time: 3.8 s


{'question': "Hi, I'm Pablo Marin. What's yours",
 'history': [],
 'output': "Hello Pablo Marin, I'm Jarvis. How can I assist you today?"}

In [16]:
printmd(agent_with_chat_history.invoke(
    {"question": "What are markov chains and is there an application in medicine?"}, 
    config=config)["output"])

Markov chains are a type of stochastic model that describes a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In other words, it is a memoryless process where the next state depends only on the current state. A Markov chain is completely described by its initial distribution and its transition probability matrix. If the distribution of the chain at time k is represented by a row vector ak, then the distribution at time k+1 is given by ak+1 = akQ, where Q is the transition probability matrix. Markov chains have applications in various fields, including medicine.

One application of Markov chains in medicine is in the prediction of transient particle transport in enclosed environments to understand the airborne infectious disease transmission risk to occupants. A combined computational fluid dynamics (CFD) and Markov chain method has been developed to predict transient particle transport in such environments. This method first calculates a transition probability matrix using CFD simulations and then applies the Markov chain technique to calculate the transient particle concentration distributions. This approach provides faster-than-real-time information about particle transport and can be used to avoid the recalculation of the particle transport equation when the source location is changed, thus reducing computing costs<sup><a href="https://doi.org/10.1111/ina.12056" target="_blank">[1]</a></sup>.

Another application is in the modeling of the spread of viruses using a Spatial Markov Chain model. This model represents a graph connecting nodes, which represent humans, and the likelihood of infectious spread from person to person is determined by the intensity of interpersonal contact. The model incorporates various lockdown scenarios and provides insights into the spread of infectious diseases<sup><a href="https://arxiv.org/pdf/2004.05635v1.pdf" target="_blank">[2]</a></sup>.

These applications demonstrate the utility of Markov chains in understanding and predicting disease transmission, particle transport, and the spread of infectious diseases in various environments.

For more details, you can refer to the provided sources.

In [17]:
try:
    printmd(agent_with_chat_history.invoke(
        {"question": "Interesting, Tell me more about the use specifically in the spread of viruses"},
        config=config)["output"])
except Exception as e:
    print(e)

The spread of viruses involves various mechanisms at the molecular, cellular, and population levels. When viruses enter the body, they can spread locally on body surfaces or through the lymphatic and blood vessels to produce systemic infection. The initial attachment and infection of cells occur on body surfaces such as the skin, mucosa of the digestive, respiratory, or urogenital tract, or the conjunctiva. Additionally, parenteral injection, either by needle or arthropod/vertebrate bites, can bypass body surfaces and lead to infection. The spread of viruses can also occur through the animal via the lymphatics, blood vessels, or nerves<sup><a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7173411/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">1</a></sup>.

Furthermore, mathematical and computer simulation modeling methods and tools have been increasingly applied to improve the understanding of disease transmission dynamics, contingency planning, and to support policy decisions on disease outbreak management. These methods and tools have been used to model the spread of zoonotic influenza viruses in animals and humans, particularly those related to the animal-human interface. The review highlights the limited application of modeling research to influenza in animals and at the animal-human interface, indicating significant gaps in knowledge regarding the transmission dynamics in animals and at the animal-human interface<sup><a href="https://doi.org/10.1111/zph.12010; https://www.ncbi.nlm.nih.gov/pubmed/22937896/?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">2</a></sup>.

In addition, the use of agent-based simulations, conceptual models, and interactive AI tools has been described in the context of managing the spread of COVID-19. These tools enable users to specify conceptual models of the impact of social distancing on the spread of COVID-19, automatically spawn agent-based simulations, and fill in simulation parameters from data. This approach provides a way to visualize simulation results and revise simulation parameters for further experimental trials or alternative conceptual models<sup><a href="https://arxiv.org/pdf/2003.13762v1.pdf?sv=2022-11-02&ss=b&srt=sco&sp=rl&se=2026-01-03T02:11:44Z&st=2024-01-02T18:11:44Z&spr=https&sig=ngrEqvqBVaxyuSYqgPVeF%2B9c0fXLs94v3ASgwg7LDBs%3D">3</a></sup>.

These insights shed light on the complex nature of virus spread and the diverse approaches used to study and manage the transmission dynamics of viruses in different contexts.

In [18]:
printmd(agent_with_chat_history.invoke({"question": "Thhank you!"}, config=config)["output"])

You're welcome! If you have any more questions or need further assistance, feel free to ask.

#### Important: there is a limitation of GPT3.5, once we start adding long prompts, and long contexts and thorough answers, or the agent makes multiple searches for multi-step questions, we run out of space!

You can minimize this by:
- Shorter System Prompt
- Smaller chunks (less than the default of 5000 characters)
- Reducing topK to bring less relevant chunks

However, you ultimately are sacrificing quality to make everything work with GPT3.5 (cheaper and faster model)

### Let's add more things we have learned so far: dynamic LLM selection of GPT4 and asyncronous streaming

In [19]:
agent = create_openai_tools_agent(llm.with_config(configurable={"model": "gpt4"}), tools, prompt) # We select now GPT-4
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
agent_with_chat_history = RunnableWithMessageHistory(agent_executor,get_session_history,input_messages_key="question", 
                                                     history_messages_key="history", history_factory_config=[userid_spec,session_id])

In prior notebooks with use the function `.stream()` of the runnable in order to stream the tokens. However if you need to stream individual tokens from the agent or surface steps occuring within tools, you would need to use a combination of `Callbacks` and `.astream()` OR the new `astream_events` API (beta).

Let’s use here the astream_events API to stream the following events:

    Agent Start with inputs
    Tool Start with inputs
    Tool End with outputs
    Stream the agent final anwer token by token
    Agent End with outputs

In [20]:
QUESTION = "Tell me more about your last answer, search again multiple times and provide a deeper explanation"

In [21]:
async for event in agent_with_chat_history.astream_events(
    {"question": QUESTION}, config=config, version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (event["name"] == "AgentExecutor"):
            print( f"Starting agent: {event['name']}")
    elif kind == "on_chain_end":
        if (event["name"] == "AgentExecutor"):  
            print()
            print("--")
            print(f"Done agent: {event['name']}")
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        # Empty content in the context of OpenAI means that the model is asking for a tool to be invoked.
        # So we only print non-empty content
        if content:
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        # print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: AgentExecutor
--
Starting tool: docsearch with inputs: {'query': 'spread of viruses'}
--
Starting tool: docsearch with inputs: {'query': 'agent-based simulations for virus spread'}
--
Starting tool: docsearch with inputs: {'query': 'mathematical modeling of virus spread'}
Done tool: docsearch
--
Done tool: docsearch
--
Done tool: docsearch
--
In the context of the spread of viruses, the behavior of viruses at the molecular and cellular levels differs from other infectious agents. Viruses enter the animal body through its surfaces and then spread either locally on body surfaces or through lymphatic and blood vessels to produce systemic infection. This process involves attachment to and infection of cells, such as the skin, the mucosa of the digestive, respiratory, or urogenital tract, or the conjunctiva. Additionally, parenteral injection, either by needle or by the bite of an arthropod or vertebrate, can bypass body surfaces, leading to infection. Viruses can remain loc

#### Note: Try to run this last question with GPT3.5 and see how you are going to run out of token space in the LLM

# Summary

We just built our first RAG BOT!.

- We learned that **Agents + Tools are the best way to go about building Bots**. <br>
- We converted the Azure Search retriever into a Tool using the function `GetDocSearchResults_Tool` in `utils.py`
- We learned about the events API (Beta), one way to stream the answer from agents
- We learned that for comprehensive, quality answers we will run out of space with GPT3.5. GPT4 then becomes necessary.


# NEXT

Now that we have a bot with one skill (Document Search), let's build more skills!. In the next Notebook, we are going to build an agent that can understand tabular data in csv file and can execute python commands